In [1]:
import pandas as pd
import numpy as np

import os
import sys

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

import keras
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint

import tensorflow as tf

import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
train_data = pd.read_csv("Train_Data.csv")
train_data.tail(100)

,ID,Office_PIN,Application_Receipt_Date,Applicant_City_PIN,Applicant_Gender,Applicant_BirthDate,Applicant_Marital_Status,Applicant_Occupation,Applicant_Qualification,Manager_DOJ,...,Manager_Status,Manager_Gender,Manager_DoB,Manager_Num_Application,Manager_Num_Coded,Manager_Business,Manager_Num_Products,Manager_Business2,Manager_Num_Products2,Business_Sourced
9427,FIN1009428,575003,6/24/2008,575002.0,F,11/9/1980,M,NaN,Graduate,6/3/2008,...,Probation,M,3/21/1980,0.0,0.0,0.0,0.0,0.0,0.0,0
9428,FIN1009429,226001,6/24/2008,226001.0,F,1/11/1961,S,Business,Graduate,5/13/2008,...,Probation,F,9/11/1973,3.0,0.0,0.0,0.0,0.0,0.0,0
9429,FIN1009430,452003,6/24/2008,450001.0,M,5/5/1982,S,Salaried,Class XII,4/29/2008,...,Probation,F,6/2/1982,4.0,0.0,0.0,0.0,0.0,0.0,0
9430,FIN1009431,520010,6/24/2008,520003.0,M,3/24/1975,M,Salaried,Graduate,12/3/2007,...,Confirmation,M,2/6/1973,1.0,1.0,412140.0,11.0,412140.0,11.0,0
9431,FIN1009432,202001,6/25/2008,202001.0,F,4/5/1986,M,NaN,Graduate,1/8/2004,...,Confirmation,M,7/2/1968,0.0,0.0,373069.0,11.0,373069.0,11.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9522,FIN1009523,160017,7/1/2008,160032.0,M,1/18/1970,M,Salaried,Graduate,5/5/2008,...,Probation,M,5/10/1967,0.0,0.0,0.0,0.0,0.0,0.0,0
9523,FIN1009524,848101,7/1/2008,848302.0,M,9/11/1956,M,NaN,Graduate,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
9524,FIN1009525,753012,7/1/2008,753014.0,F,8/7/1975,M,Salaried,Graduate,8/22/2006,...,Confirmation,M,7/22/1970,0.0,0.0,316126.0,9.0,305775.0,8.0,0
9525,FIN1009526,575003,7/1/2008,571248.0,M,12/23/1986,S,Salaried,Class XII,6/5/2008,...,Probation,M,9/23/1976,0.0,0.0,0.0,0.0,0.0,0.0,0


In [3]:
test_data = pd.read_csv("Test_Data.csv")
test_data.tail(100)

,ID,Office_PIN,Application_Receipt_Date,Applicant_City_PIN,Applicant_Gender,Applicant_BirthDate,Applicant_Marital_Status,Applicant_Occupation,Applicant_Qualification,Manager_DOJ,...,Manager_Grade,Manager_Status,Manager_Gender,Manager_DoB,Manager_Num_Application,Manager_Num_Coded,Manager_Business,Manager_Num_Products,Manager_Business2,Manager_Num_Products2
4945,FIN1014473,751009,12/31/2008,754119.0,F,6/7/1972,S,NaN,Graduate,12/18/2008,...,4.0,Probation,F,7/11/1972,0.0,0.0,0.0,0.0,0.0,0.0
4946,FIN1014474,680001,1/1/2009,680587.0,M,6/7/1950,M,NaN,Graduate,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4947,FIN1014475,641018,1/1/2009,641012.0,M,5/9/1971,M,Business,Graduate,12/2/2008,...,4.0,Probation,M,1/8/1974,0.0,0.0,0.0,0.0,0.0,0.0
4948,FIN1014476,273001,1/1/2009,274407.0,M,1/3/1983,M,Business,Graduate,12/9/2008,...,3.0,Probation,M,8/7/1976,0.0,0.0,0.0,0.0,0.0,0.0
4949,FIN1014477,530003,1/1/2009,530028.0,F,4/6/1974,M,NaN,Graduate,12/3/2008,...,2.0,Probation,M,7/13/1972,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5040,FIN1014568,691008,1/1/2009,691601.0,F,5/21/1966,M,Others,Graduate,12/2/2008,...,3.0,Probation,M,5/27/1978,0.0,0.0,0.0,0.0,0.0,0.0
5041,FIN1014569,691008,1/1/2009,691601.0,M,12/2/1949,M,Others,Graduate,12/2/2008,...,3.0,Probation,M,5/27/1978,0.0,0.0,0.0,0.0,0.0,0.0
5042,FIN1014570,691008,1/1/2009,691005.0,M,5/20/1977,M,Business,Class XII,12/2/2008,...,3.0,Probation,M,5/27/1978,0.0,0.0,0.0,0.0,0.0,0.0
5043,FIN1014571,208001,1/1/2009,208022.0,M,7/22/1985,M,NaN,Graduate,9/25/2008,...,3.0,Probation,M,6/26/1978,4.0,1.0,15347.0,1.0,15347.0,1.0


In [4]:
train_data.info

<bound method DataFrame.info of               ID  Office_PIN Application_Receipt_Date  Applicant_City_PIN  \
0     FIN1000001      842001                4/16/2007            844120.0   
1     FIN1000002      842001                4/16/2007            844111.0   
2     FIN1000003      800001                4/16/2007            844101.0   
3     FIN1000004      814112                4/16/2007            814112.0   
4     FIN1000005      814112                4/16/2007            815351.0   
...          ...         ...                      ...                 ...   
9522  FIN1009523      160017                 7/1/2008            160032.0   
9523  FIN1009524      848101                 7/1/2008            848302.0   
9524  FIN1009525      753012                 7/1/2008            753014.0   
9525  FIN1009526      575003                 7/1/2008            571248.0   
9526  FIN1009527      411006                 7/1/2008            411006.0   

     Applicant_Gender Applicant_BirthDate A

In [5]:
train_data.isnull().sum()

ID                                0
Office_PIN                        0
Application_Receipt_Date          0
Applicant_City_PIN               97
Applicant_Gender                 67
Applicant_BirthDate              73
Applicant_Marital_Status         73
Applicant_Occupation           1221
Applicant_Qualification          86
Manager_DOJ                     683
Manager_Joining_Designation     683
Manager_Current_Designation     683
Manager_Grade                   683
Manager_Status                  683
Manager_Gender                  683
Manager_DoB                     683
Manager_Num_Application         683
Manager_Num_Coded               683
Manager_Business                683
Manager_Num_Products            683
Manager_Business2               683
Manager_Num_Products2           683
Business_Sourced                  0
dtype: int64

In [6]:
train_data['Manager_Num_Products2'].fillna(7, inplace = True)
train_data['Manager_Business2'].fillna(1.829263e+05, inplace = True)
train_data['Manager_Num_Products'].fillna(7, inplace = True)
train_data['Manager_Business'].fillna(1.843710e+05, inplace = True)
train_data['Manager_Num_Coded'].fillna(0.758933, inplace = True)
train_data['Manager_Grade'].fillna(3, inplace = True)
train_data['Manager_Num_Application'].fillna(2, inplace = True)

In [7]:
test_data['Manager_Num_Products2'].fillna(3.5, inplace = True)
test_data['Manager_Business2'].fillna(9.895073e+04, inplace = True)
test_data['Manager_Num_Products'].fillna(4, inplace = True)
test_data['Manager_Business'].fillna(1.115425e+05, inplace = True)
test_data['Manager_Num_Coded'].fillna(1, inplace = True)
test_data['Manager_Grade'].fillna(3, inplace = True)
test_data['Manager_Num_Application'].fillna(2.5, inplace = True)

In [9]:
train_data['Applicant_Qualification'].dropna(inplace = True)
train_data['Applicant_Occupation'].dropna(inplace = True)
train_data['Manager_Current_Designation'].dropna(inplace = True)
train_data['Manager_Joining_Designation'].dropna(inplace = True)
train_data['Manager_Status'].dropna(inplace = True)

In [10]:
train_data.info


<bound method DataFrame.info of               ID  Office_PIN Application_Receipt_Date  Applicant_City_PIN  \
0     FIN1000001      842001                4/16/2007            844120.0   
1     FIN1000002      842001                4/16/2007            844111.0   
2     FIN1000003      800001                4/16/2007            844101.0   
3     FIN1000004      814112                4/16/2007            814112.0   
4     FIN1000005      814112                4/16/2007            815351.0   
...          ...         ...                      ...                 ...   
9522  FIN1009523      160017                 7/1/2008            160032.0   
9523  FIN1009524      848101                 7/1/2008            848302.0   
9524  FIN1009525      753012                 7/1/2008            753014.0   
9525  FIN1009526      575003                 7/1/2008            571248.0   
9526  FIN1009527      411006                 7/1/2008            411006.0   

     Applicant_Gender Applicant_BirthDate A

In [31]:
# Getting Features
features = ['Manager_Num_Products2','Manager_Num_Products','Manager_Business','Manager_Business2','Manager_Num_Coded', 'Manager_Num_Application','Manager_Grade','Manager_Status','Manager_Current_Designation','Applicant_Occupation']
x = pd.get_dummies(train_data[features])

# Predicting value
y = train_data['Business_Sourced']
y.shape, x.shape

((9527,), (9527, 19))

In [32]:
x

,Manager_Num_Products2,Manager_Num_Products,Manager_Business,Manager_Business2,Manager_Num_Coded,Manager_Num_Application,Manager_Grade,Manager_Status_Confirmation,Manager_Status_Probation,Manager_Current_Designation_Level 1,Manager_Current_Designation_Level 2,Manager_Current_Designation_Level 3,Manager_Current_Designation_Level 4,Manager_Current_Designation_Level 5,Applicant_Occupation_Business,Applicant_Occupation_Others,Applicant_Occupation_Salaried,Applicant_Occupation_Self Employed,Applicant_Occupation_Student
0,28.0,28.0,335249.0,335249.0,1.000000,2.0,3.0,1,0,0,1,0,0,0,0,1,0,0,0
1,28.0,28.0,335249.0,335249.0,1.000000,2.0,3.0,1,0,0,1,0,0,0,0,1,0,0,0
2,24.0,24.0,357184.0,357184.0,0.000000,0.0,2.0,1,0,1,0,0,0,0,1,0,0,0,0
3,22.0,22.0,318356.0,318356.0,0.000000,0.0,4.0,1,0,0,0,1,0,0,0,0,1,0,0
4,17.0,17.0,230402.0,230402.0,1.000000,2.0,2.0,1,0,1,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9522,0.0,0.0,0.0,0.0,0.000000,0.0,3.0,0,1,0,1,0,0,0,0,0,1,0,0
9523,7.0,7.0,184371.0,182926.3,0.758933,2.0,3.0,0,0,0,0,0,0,0,0,0,0,0,0
9524,8.0,9.0,316126.0,305775.0,0.000000,0.0,3.0,1,0,0,1,0,0,0,0,0,1,0,0
9525,0.0,0.0,0.0,0.0,0.000000,0.0,4.0,0,1,0,0,1,0,0,0,0,1,0,0


In [33]:
xtest = pd.get_dummies(test_data[features])
xtest

,Manager_Num_Products2,Manager_Num_Products,Manager_Business,Manager_Business2,Manager_Num_Coded,Manager_Num_Application,Manager_Grade,Manager_Status_Confirmation,Manager_Status_Probation,Manager_Current_Designation_Level 1,Manager_Current_Designation_Level 2,Manager_Current_Designation_Level 3,Manager_Current_Designation_Level 4,Manager_Current_Designation_Level 5,Applicant_Occupation_Business,Applicant_Occupation_Others,Applicant_Occupation_Salaried,Applicant_Occupation_Self Employed,Applicant_Occupation_Student
0,0.0,0.0,0.0,0.00,0.0,1.0,2.0,0,1,1,0,0,0,0,0,0,1,0,0
1,0.0,0.0,0.0,0.00,0.0,2.0,2.0,0,1,1,0,0,0,0,0,0,1,0,0
2,3.5,4.0,111542.5,98950.73,1.0,2.5,3.0,0,0,0,0,0,0,0,0,0,0,0,0
3,3.5,4.0,111542.5,98950.73,1.0,2.5,3.0,0,0,0,0,0,0,0,0,0,1,0,0
4,3.0,3.0,97352.0,97352.00,1.0,4.0,3.0,1,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5040,0.0,0.0,0.0,0.00,0.0,0.0,3.0,0,1,0,1,0,0,0,0,1,0,0,0
5041,0.0,0.0,0.0,0.00,0.0,0.0,3.0,0,1,0,1,0,0,0,0,1,0,0,0
5042,0.0,0.0,0.0,0.00,0.0,0.0,3.0,0,1,0,1,0,0,0,1,0,0,0,0
5043,1.0,1.0,15347.0,15347.00,1.0,4.0,3.0,0,1,0,1,0,0,0,0,0,0,0,0


In [35]:
encoder = OneHotEncoder()
y = encoder.fit_transform(np.array(y).reshape(-1,1)).toarray()
print(y.shape,x.shape,xtest.shape)

# scaling our data with sklearn's Standard scaler
scaler = StandardScaler()
x = scaler.fit_transform(x)
xtest = scaler.transform(xtest)
x.shape, y.shape, xtest.shape

(9527, 2) (9527, 19) (5045, 19)


((9527, 19), (9527, 2), (5045, 19))

In [36]:
# making our data compatible to model.
x = np.expand_dims(x, axis=2)
xtest = np.expand_dims(xtest, axis=2)

print(y.shape)
print(x.shape)
print(xtest.shape)

model = Sequential()
model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu', input_shape=(x.shape[1], 1)))
model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

model.add(Conv1D(128, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))
model.add(Dropout(0.2))

model.add(Conv1D(64, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

model.add(Flatten())
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(units=2, activation='softmax'))
model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

model.summary()

(9527, 2)
(9527, 19, 1)
(5045, 19, 1)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 19, 256)           1536      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 10, 256)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 10, 256)           327936    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 5, 256)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 5, 128)            163968    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 3, 128)            0         
_________________________________________________________________
dropout (Dropout) 

In [37]:
rlrp = ReduceLROnPlateau(monitor='loss', factor=0.4, verbose=0, patience=2, min_lr=0.0000001)
model.fit(x, y, batch_size=64, epochs=100, callbacks=[rlrp])

Epoch 1/100
149/149 [==============================] - 3s 21ms/step - loss: 0.6528 - accuracy: 0.6551
Epoch 2/100
149/149 [==============================] - 3s 20ms/step - loss: 0.6427 - accuracy: 0.6572
Epoch 3/100
149/149 [==============================] - 3s 20ms/step - loss: 0.6379 - accuracy: 0.6569
Epoch 4/100
149/149 [==============================] - 3s 20ms/step - loss: 0.6338 - accuracy: 0.6571
Epoch 5/100
149/149 [==============================] - 3s 20ms/step - loss: 0.6351 - accuracy: 0.6571
Epoch 6/100
149/149 [==============================] - 4s 25ms/step - loss: 0.6334 - accuracy: 0.6562
Epoch 7/100
149/149 [==============================] - 4s 25ms/step - loss: 0.6315 - accuracy: 0.6567
Epoch 8/100
149/149 [==============================] - 4s 26ms/step - loss: 0.6316 - accuracy: 0.6568
Epoch 9/100
149/149 [==============================] - 4s 26ms/step - loss: 0.6296 - accuracy: 0.6561
Epoch 10/100
149/149 [==============================] - 4s 26ms/step - loss: 0.628

In [38]:
pred = model.predict(xtest)
y_pred = encoder.inverse_transform(pred)
predictions = y_pred.flatten()

predictions

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [39]:
output = pd.DataFrame({'ID': test_data.ID, 'Business_Sourced': predictions.astype(int)})
output.to_csv('submission2.csv', index=False)

In [40]:
output.head(100)

,ID,Business_Sourced
0,FIN1009528,0
1,FIN1009529,0
2,FIN1009530,0
3,FIN1009531,0
4,FIN1009532,0
...,...,...
95,FIN1009623,0
96,FIN1009624,0
97,FIN1009625,0
98,FIN1009626,0


In [41]:
output

,ID,Business_Sourced
0,FIN1009528,0
1,FIN1009529,0
2,FIN1009530,0
3,FIN1009531,0
4,FIN1009532,0
...,...,...
5040,FIN1014568,0
5041,FIN1014569,0
5042,FIN1014570,0
5043,FIN1014571,0
